In [1]:
from functions import *

Instala a biblioteca
```
pip install deltalake 
```

In [2]:
import pandas as pd
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
import deltalake

In [3]:
# le um arquivo csv e converte para dataframe
df = pd.read_csv("csv\Data7602DescendingYearOrder\parte1.csv")

In [4]:
df.head()

,anzsic06,Area,year,geo_count,ec_count,dataset
0,A,A100100,2022,93,190,dataset_01
1,A,A100200,2022,138,190,dataset_01
2,A,A100300,2022,6,25,dataset_01
3,A,A100400,2022,57,50,dataset_01
4,A,A100500,2022,57,95,dataset_01


In [5]:
df.shape

(2992677, 6)

Transforma o dataframe em um **data lakehouse** !!!

In [6]:
write_deltalake(
    table_or_uri = 'delta_table', # URI of a table or a DeltaTable object.
    data = df,                    # Data to write
    mode = 'overwrite',
    description = 'dataframe created by a pandas dataframe',
    partition_by = ['year'])      # List of columns to partition the table by.

Ao executar o write_deltalake, é criado uma pasta conforme passado para o parametro table_or_uri. Na pasta são criadas pastas filhas particionadas pelo parâmetro partition_by, com arquivos .parquet.

In [7]:
# esquema da pasta criada

print(print_directory_tree('delta_table'))

delta_table
  delta_table\year=2011
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2012
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2013
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2014
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2015
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2016
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2017
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2018
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2019
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2020
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2021
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\year=2022
    0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet
  delta_table\_delta_log
    00000000000000000000.json
None


In [8]:
DeltaTable('delta_table', version = 0).to_pandas()

,anzsic06,Area,year,geo_count,ec_count,dataset
0,A,A100100,2011,87,200,dataset_01
1,A,A100200,2011,171,130,dataset_01
2,A,A100300,2011,18,15,dataset_01
3,A,A100400,2011,57,35,dataset_01
4,A,A100500,2011,75,140,dataset_01
...,...,...,...,...,...,...
2992672,M70,A183000,2013,3,0,dataset_01
2992673,M70,A183200,2013,3,3,dataset_01
2992674,M70,A183500,2013,6,6,dataset_01
2992675,M70,A183600,2013,0,0,dataset_01


**Metadados**

In [9]:
DeltaTable('delta_table').schema().to_pyarrow()

anzsic06: string
Area: string
year: int64
geo_count: int64
ec_count: int64
dataset: string

In [10]:
DeltaTable('delta_table').history()

[{'timestamp': 1692909223890,
  'operation': 'CREATE TABLE',
  'operationParameters': {'metadata': '{"configuration":{},"created_time":1692909223889,"description":"dataframe created by a pandas dataframe","format":{"options":{},"provider":"parquet"},"id":"265bb84c-3629-4579-9dff-88959bfc7770","name":null,"partition_columns":["year"],"schema":{"fields":[{"metadata":{},"name":"anzsic06","nullable":true,"type":"string"},{"metadata":{},"name":"Area","nullable":true,"type":"string"},{"metadata":{},"name":"year","nullable":true,"type":"long"},{"metadata":{},"name":"geo_count","nullable":true,"type":"long"},{"metadata":{},"name":"ec_count","nullable":true,"type":"long"},{"metadata":{},"name":"dataset","nullable":true,"type":"string"}],"type":"struct"}}',
   'protocol': '{"minReaderVersion":1,"minWriterVersion":1}',
   'location': 'file:///C:/Users/pjesus/Documents/delta/delta_table',
   'mode': 'ErrorIfExists'},
  'clientVersion': 'delta-rs.0.13.1'}]

In [11]:
DeltaTable('delta_table').get_add_actions(flatten=True).to_pandas().head(2)

,path,size_bytes,modification_time,data_change,partition.year,num_records,null_count.anzsic06,min.anzsic06,max.anzsic06,null_count.Area,...,max.Area,null_count.geo_count,min.geo_count,max.geo_count,null_count.ec_count,min.ec_count,max.ec_count,null_count.dataset,min.dataset,max.dataset
0,year=2011/0-5997f68b-3b99-4618-84d4-54746e3e51...,109820,2023-08-24 20:33:43.742,True,2011,36907,0,A,C203,0,...,TTotal,0,0,74712,0,0,214600,0,dataset_01,dataset_01
1,year=2021/0-5997f68b-3b99-4618-84d4-54746e3e51...,823158,2023-08-24 20:33:43.793,True,2021,274328,0,A,Total,0,...,TTotal,0,0,600372,0,0,2314700,0,dataset_01,dataset_01


**Time Travel**

In [12]:
# dataframe com mesmo schema do df1
df2 = pd.read_csv('csv\Data7602DescendingYearOrder\parte2.csv')
df.head(6)

,anzsic06,Area,year,geo_count,ec_count,dataset
0,A,A100100,2022,93,190,dataset_01
1,A,A100200,2022,138,190,dataset_01
2,A,A100300,2022,6,25,dataset_01
3,A,A100400,2022,57,50,dataset_01
4,A,A100500,2022,57,95,dataset_01
5,A,A100600,2022,12,30,dataset_01


In [13]:
write_deltalake('delta_table', df2,
        mode = 'append',
        partition_by= ['year'])

In [14]:
DeltaTable('delta_table', version = 0).to_pandas()

,anzsic06,Area,year,geo_count,ec_count,dataset
0,A,A100100,2011,87,200,dataset_01
1,A,A100200,2011,171,130,dataset_01
2,A,A100300,2011,18,15,dataset_01
3,A,A100400,2011,57,35,dataset_01
4,A,A100500,2011,75,140,dataset_01
...,...,...,...,...,...,...
2992672,M70,A183000,2013,3,0,dataset_01
2992673,M70,A183200,2013,3,3,dataset_01
2992674,M70,A183500,2013,6,6,dataset_01
2992675,M70,A183600,2013,0,0,dataset_01


In [15]:
DeltaTable('delta_table', version = 1).to_pandas()

,anzsic06,Area,year,geo_count,ec_count,dataset
0,A,A100100,2011,87,200,dataset_01
1,A,A100200,2011,171,130,dataset_01
2,A,A100300,2011,18,15,dataset_01
3,A,A100400,2011,57,35,dataset_01
4,A,A100500,2011,75,140,dataset_01
...,...,...,...,...,...,...
5984854,Total,A322700,2000,93,380,dataset_02
5984855,Total,A322900,2000,18,12,dataset_02
5984856,Total,A323000,2000,435,2150,dataset_02
5984857,Total,A323100,2000,195,280,dataset_02


**Schema Reiforcement**

In [16]:
DeltaTable('delta_table').schema().to_pyarrow()

anzsic06: string
Area: string
year: int64
geo_count: int64
ec_count: int64
dataset: string

In [17]:
df3 = pd.read_csv('csv\Data7602DescendingYearOrder\parte3.csv')

df3['geo_count'] = df3['geo_count'].astype(float)
df3['ec_count']  = df3['ec_count'].astype(float)
df3['year']  = df3['year'].astype(float)

Schema do df3 diferente do delta table

In [18]:
df3.dtypes

anzsic06      object
Area          object
year         float64
geo_count    float64
ec_count     float64
dataset       object
dtype: object

In [19]:
try:
    write_deltalake('delta_table', df3,
                mode = 'append',
                partition_by= ['year'])

except ValueError as error:
    print(error)

Schema of data does not match table schema
Table schema:
anzsic06: string
Area: string
year: double
geo_count: double
ec_count: double
dataset: string
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 942
Data Schema:
anzsic06: string
Area: string
year: int64
geo_count: int64
ec_count: int64
dataset: string


**Changed Data Feed**

In [20]:
currently_version = DeltaTable("delta_table").version()
currently_version

1

In [21]:
DeltaTable("delta_table", version = 0).file_uris()

['C:/Users/pjesus/Documents/delta/delta_table/year=2011/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2021/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2022/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2012/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2019/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2020/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2016/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2014/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2018/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/del

In [22]:
DeltaTable("delta_table", version = 1).file_uris()

['C:/Users/pjesus/Documents/delta/delta_table/year=2011/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2021/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2022/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2012/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2019/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2020/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2016/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2014/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/delta_table/year=2018/0-5997f68b-3b99-4618-84d4-54746e3e5173-0.parquet',
 'C:/Users/pjesus/Documents/delta/del